In [2]:
# Initial imports
import panel as pn
pn.extension('plotly')

import datetime as dt
import hvplot.pandas
import json
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import seaborn as sns
import time

from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

## Part 1: Trading Volume Data

Coinbase provides historical data for their products via their [public API](https://docs.pro.coinbase.com). For the trading volume analysis, the following endpoints are queried: `/currencies` and `/products`.

In [33]:
# Coinbase API endpoints
coinbase_api_url = "https://api.pro.coinbase.com"
cb_products_api_url = f"{coinbase_api_url}/products"
cb_currencies_api_url = f"{coinbase_api_url}/currencies"

In [34]:
# Get currencies (fiat and crypto)
cb_currencies_response = requests.get(cb_currencies_api_url)
cb_currencies = cb_currencies_response.json()
cb_currencies_df = pd.DataFrame(cb_currencies)
cb_currencies_df = pd.concat([cb_currencies_df, pd.io.json.json_normalize(cb_currencies_df["details"])], axis=1).drop(columns=["details"])
cb_currencies_df.head()

,convertible_to,id,max_precision,message,min_size,name,status,crypto_address_link,crypto_transaction_link,display_name,group_types,max_withdrawal_amount,min_withdrawal_amount,network_confirmations,processing_time_seconds,push_payment_methods,sort_order,symbol,type
0,[],ALGO,0.000001,,1,Algorand,online,https://algoexplorer.io/address/{{address}},https://algoexplorer.io/tx/{{txId}},,[],1000000.0,0.1000,1,5,[crypto],93,A,crypto
1,[],DASH,0.00000001,,1,Dash,online,https://chain.so/address/DASH/{{address}},https://chain.so/tx/DASH/{{address}},,[],10000.0,0.0001,2,0,[crypto],47,,crypto
2,[],OXT,0.00000001,,1,Orchid,online,https://etherscan.io/token/0x4575f41308EC1483f...,https://etherscan.io/tx/0x{{txId}},,[],1000000.0,0.0100,35,0,[crypto],48,,crypto
3,[],ATOM,0.000001,,1,Cosmos,online,https://cosmos.bigdipper.live/account/{{address}},https://cosmos.bigdipper.live/transactions/{{t...,,[],100000.0,0.1000,0,5,[crypto],51,,crypto
4,[],KNC,0.00000001,,1,Kyber Network,online,https://etherscan.io/token/0xdd974d5c2e2928dea...,https://etherscan.io/tx/0x{{txId}},,[],2156253.0,0.0100,35,0,[crypto],120,Ξ,crypto


In [35]:
# Get Coinbase products
cb_products_response = requests.get(cb_products_api_url)
cb_products = cb_products_response.json()
cb_products_df = pd.DataFrame(cb_products)
cb_products_df.head()

,base_currency,base_increment,base_max_size,base_min_size,cancel_only,display_name,id,limit_only,margin_enabled,max_market_funds,min_market_funds,post_only,quote_currency,quote_increment,status,status_message,trading_disabled
0,ALGO,1.00000000,500000.00000000,1.00000000,False,ALGO/EUR,ALGO-EUR,False,False,100000,10,False,EUR,0.00010000,online,,False
1,EOS,0.10000000,50000.00000000,0.10000000,False,EOS/EUR,EOS-EUR,False,False,100000,10,False,EUR,0.00100000,online,,False
2,NU,1.00000000,1300000.00000000,10.00000000,False,NU/BTC,NU-BTC,False,False,6,0.0001,False,BTC,0.00000001,online,,False
3,REP,0.00000100,5000.00000000,0.10000000,False,REP/USD,REP-USD,False,False,30000,10,False,USD,0.01000000,online,,False
4,GRT,0.01000000,2500000.00000000,10.00000000,False,GRT/BTC,GRT-BTC,False,False,5.2,0.0000001,False,BTC,0.00000001,online,,False


In [10]:
def get_date_list(start_date, end_date, interval=300):
    """
    Get the list of dates in 300-day incremenets.
    """
    delta_days = (end_date - start_date).days
    iterations = math.floor(delta_days / interval) + 1
    date_list = [start_date + dt.timedelta(days=x*interval) for x in range(iterations)]
    return date_list

In [47]:
def get_coinbase_historical_data(id, start_date):
    """
    Get historical data (daily) for a product between the start and end dates.
    """
    start_str = dt.datetime.strftime(start_date, "%Y-%m-%d")
    end_str = dt.datetime.strftime(start_date + dt.timedelta(days=300), "%Y-%m-%d")
    candles_url = f"{products_api_url}/{id}/candles?start={start_str}&end={end_str}&granularity=86400"
    candles_response = requests.get(candles_url)
    print(f"{id}: {start_str} to {end_str}, Status: {candles_response.status_code}")
    candles = candles_response.json()
    candles_df = pd.DataFrame(candles, columns=["time", "low", "high", "open", "close", "volume"])
    return candles_df

In [51]:
# Get list of fiat and cryptocurrencies
cb_crypto_currencies = cb_currencies_df[cb_currencies_df["type"] == "crypto"]["id"].tolist()
cb_fiat_currencies = cb_currencies_df[cb_currencies_df["type"] == "fiat"]["id"].tolist()
# Find Coinbase products bought/sold with fiat currency
cb_fiat_products_df = cb_products_df[cb_products_df["quote_currency"].isin(cb_fiat_currencies)]

print(f"Crypto currencies: {cb_crypto_currencies}\n")
print(f"Fiat currencies: {cb_fiat_currencies}\n")
print(f"{len(cb_fiat_products_df.index)} Coinbase products bought/sold with fiat currency")

Crypto currencies: ['ALGO', 'DASH', 'OXT', 'ATOM', 'KNC', 'XRP', 'REP', 'MKR', 'CGLD', 'COMP', 'NMR', 'OMG', 'BAND', 'UMA', 'XLM', 'EOS', 'ZRX', 'BAT', 'LOOM', 'UNI', 'YFI', 'LRC', 'CVC', 'DNT', 'MANA', 'GNT', 'REN', 'LINK', 'BTC', 'BAL', 'LTC', 'ETH', 'BCH', 'ETC', 'USDC', 'ZEC', 'XTZ', 'DAI', 'WBTC', 'NU', 'FIL', 'AAVE', 'SNX', 'BNT', 'GRT']

Fiat currencies: ['EUR', 'GBP', 'USD']

78 Coinbase products bought/sold with fiat currency


In [52]:
# Get list of dates to fetch data for in 300-day increments
today = dt.datetime.today()
base = dt.datetime.strptime("2009-01-01", "%Y-%m-%d")
cb_date_list = get_date_list(base, today)

# Get data from Coinbase and append to dataframe
trading_volume_data_df = pd.DataFrame()
for idx, row in cb_fiat_products_df.iterrows():
    product_id = row["id"]
    base_currency = row["base_currency"]
    quote_currency = row["quote_currency"]
    for start_date in cb_date_list:
        historical_df = get_coinbase_historical_data(product_id, start_date)
        if not df.empty:
            historical_df["base_currency"] = base_currency
            historical_df["quote_currency"] = quote_currency
            trading_volume_data_df = trading_volume_data_df.append(historical_df)
        time.sleep(2)

trading_volume_data_df.head()

SD: 2017-03-20 to 2018-01-14, Status: 200
BTC-USD: 2018-01-14 to 2018-11-10, Status: 200
BTC-USD: 2018-11-10 to 2019-09-06, Status: 200
BTC-USD: 2019-09-06 to 2020-07-02, Status: 200
BTC-USD: 2020-07-02 to 2021-04-28, Status: 200
BTC-GBP: 2009-01-01 to 2009-10-28, Status: 200
BTC-GBP: 2009-10-28 to 2010-08-24, Status: 200
BTC-GBP: 2010-08-24 to 2011-06-20, Status: 200
BTC-GBP: 2011-06-20 to 2012-04-15, Status: 200
BTC-GBP: 2012-04-15 to 2013-02-09, Status: 200
BTC-GBP: 2013-02-09 to 2013-12-06, Status: 200
BTC-GBP: 2013-12-06 to 2014-10-02, Status: 200
BTC-GBP: 2014-10-02 to 2015-07-29, Status: 200
BTC-GBP: 2015-07-29 to 2016-05-24, Status: 200
BTC-GBP: 2016-05-24 to 2017-03-20, Status: 200
BTC-GBP: 2017-03-20 to 2018-01-14, Status: 200
BTC-GBP: 2018-01-14 to 2018-11-10, Status: 200
BTC-GBP: 2018-11-10 to 2019-09-06, Status: 200
BTC-GBP: 2019-09-06 to 2020-07-02, Status: 200
BTC-GBP: 2020-07-02 to 2021-04-28, Status: 200
NU-USD: 2009-01-01 to 2009-10-28, Status: 200
NU-USD: 2009-10-28 

,time,low,high,open,close,volume,base_currency,quote_currency
0,1611446400,0.4563,0.4783,0.4572,0.4699,212031,ALGO,EUR
1,1611360000,0.4128,0.4705,0.4191,0.4575,2312202,ALGO,EUR
2,1611273600,0.3528,0.4438,0.3804,0.4188,2636644,ALGO,EUR
3,1611187200,0.3691,0.4533,0.4526,0.3793,1693635,ALGO,EUR
4,1611100800,0.4010,0.4530,0.4317,0.4525,3203612,ALGO,EUR


In [53]:
# convert epoch time to date
trading_volume_data_df = trading_volume_data_df.drop_duplicates()
trading_volume_data_df["date"] = pd.to_datetime(trading_volume_data_df["time"], unit="s")
trading_volume_data_df = trading_volume_data_df.drop(columns=["time"])

In [54]:
# Export dataframe to CSV for faster loading
cb_data_filepath = Path("../data/coinbase_historical_data.csv")
trading_volume_data_df.to_csv(cb_data_filepath, index=False)